# Labolatorium 4
## Wyszukiwarka

---
### 1. Przygotowanie zbioru (> 1000 elementów) dokumentów tekstowych w języku angielskim.

Dokumenty znajdują się w folderze `articles`

---
### 2. & 3. Określenie słów kluczowych (termów) potrzebnych do wyznaczenia *bag-of-words* dla każdego dokumentów oraz wyznaczenie *bag-of-words* dla każdego dokumentu $d_j$

W naszym przypadku bazą termów będzie unia wszystkich słów występujących we wszystkich tekstach. Postaramy się pominąć wszelkiego rodzaju znaki interpunkcyjne oraz często występujące słowa w języku angielskim, które nie mają większego znaczenia (np. 'a', 'this', 'of').

Przydatne importy:

In [188]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import os
import string
import scipy.sparse as sparse

try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download('stopwords')

#### zapisywanie i odczyt najważniejszych struktur z pliku

In [189]:
def save_counter_to_file(file_name, bag_of_words):
    text = ""
    for word, count in bag_of_words.items():
        text = text + word + " " + str(count) + "\n"
    filepath = os.path.join(os.getcwd(), file_name)
    with open(filepath, "w", encoding="utf-8") as article_file:
        article_file.write(text)

        
def load_counter_from_file(file_name):
    file_path = os.path.join(os.getcwd(), file_name)

    bag_of_words = Counter()

    
    with open(file_path, "r", encoding="utf8") as text:
            for line in text:
                elems = line.split()  # elems[0] - word, elems[1] - word count
                bag_of_words[elems[0]] = int(elems[1])
                
    return bag_of_words

#### Funkcja, która na podstawie tekstu tworzy wektor *bag-of-words*

In [190]:
def get_bag_of_words_from_text(text):
    # make all article lowercase
    text = text.lower()

    # divide into words (it still includes punctuation)
    words = [word for sentence in sent_tokenize(text) for word in word_tokenize(sentence)]

    # remove meaningless English words such as 'the', 'a' etc.
    english_stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in english_stop_words]

    # remove punctuation
    punctuation = set(string.punctuation)
    punctuation.add("...")
    words = [word for word in words if word not in punctuation]

    # stem words - this makes words with the same meaning equal, e.g. responsibility, responsible => respons
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # remove meaningless 1 or 2 chars words
    words = [word for word in words if len(word) > 2]

    # create a bag_of_words based on this article
    bag_of_words = Counter(words)

    return bag_of_words

#### funkcja, która dla każdego artykułu:
- wczytuje jego tekst i na jego podstawie tworzy wektor cech *bag-of-words* ${d_j}$
- Zapisuje ten wektor do odpowiedniego pliku w katalogu `bags`

W końcu na podstawie wszystkich *bag-of-words* tworzy słownik bazowy dla wszystkich dokumentów i zapisuje go do pliku `base_dictionary`

In [191]:
def process_articles(source_directory = "articles"):
    base_dictionary = Counter()
    
    curr_path = os.getcwd()
    source_path = os.path.join(curr_path, source_directory)
    bags_dir = os.path.join(curr_path, "bags")
    if not os.path.exists(bags_dir):
        os.makedirs(bags_dir)
        
    for file_name in os.listdir(source_path):
        file_path = os.path.join(source_path, file_name)

        with open(file_path, "r", encoding="utf8") as text:

            bag_of_words = get_bag_of_words_from_text(text.read())
            base_dictionary += bag_of_words
            save_counter_to_file(f"bags/{file_name}", bag_of_words)
    
    save_counter_to_file("base_dictionary", base_dictionary)

---
### 4. Budowanie rzadkiej macierzy wektorów cech *term-by-document matrix*.
Wektory ułożone są kolumnowo $A_{m \times n} = [d_1|d_2|...|d_n]$ (m jest liczbą termów w słowniku, n jest liczbą dokumentów).

Poniższa funkcja tworzy macierz cech (pierwsza zwracana wartość), ponadtdo zwraca opis osi Y (termy w odpowiedniej kolejności) jako drugą wartość oraz opis osi X (nazwy dokumentów w odpowiedniej kolejności) jako trzecią wartość

In [192]:
def build_term_by_document_matrix(base_ditionary_name = "base_dictionary", bags_dir = "bags"):
    base_dictionary = load_counter_from_file(base_ditionary_name)
    terms_list = list(base_dictionary)
    
    
    bag_names = os.listdir(bags_dir)
    N = len(bag_names)
    M = len(terms_list)
    
    term_by_document_matrix = sparse.lil_matrix((M, N))

    for j, file_name in enumerate(bag_names):
        bag_of_terms = load_counter_from_file(os.path.join(bags_dir, file_name))

        
        for i, term in enumerate(terms_list):
            term_by_document_matrix[i, j] = bag_of_terms[term]
            
    
    return term_by_document_matrix.tocsr(), terms_list, bag_names

---
### 5. Przetworzenie wstępnie otrzymanego zbioru danych przez *inverse document frequency*. 

$ IDF(w) = log(\frac{N}{n_w}) $

gdzie:\
$n_w$ - liczba dokumentów, w których występuje słowo $w$\
$N$ - całkowita liczba dokumentów

In [193]:
def IDF(row):
    N = row.shape[1]
    n_w = row.count_nonzero()
    return np.log(N / n_w)

def tf_to_idf(term_by_document_matrix):
    N = term_by_document_matrix.shape[0]
    idfs = np.zeros((N))
    for i in range(N):
        idfs[i] = IDF(term_by_document_matrix[i,:])
        term_by_document_matrix[i,:] *= idfs[i]
    return term_by_document_matrix, idfs

---
### Czas przetestować wyżej napisane funkcj i utworzyć pełnoprawną macierz *term-by-document*

In [194]:
process_articles()
init_matrix, ax_Y, ax_X = build_term_by_document_matrix()
idf_matrix, idfs = tf_to_idf(init_matrix)
idf_matrix.todense()

matrix([[180.03779932,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [ 78.78699934,   0.        ,   0.        , ...,   8.75411104,
           0.        ,   0.        ],
        [  4.46303042,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           7.23561914,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           7.23561914,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
          14.47123828,   0.        ]])

---
### 6. Program pozwalający na wprowadzenie zapytania, który następnie przekształci je do reprezentacji wektorowej $q$

#### Program ma zwrócić $k$ dokumentów najbardziej zbliżonych do podanego zapytania $q$.

Należy użyć korelacji między wektorami jako miary podobieństwa:

$cos(\theta_j) = \frac{q^T d_j}{||q||\cdot||d_j||} = \frac{q^T Ae_j}{||q||\cdot||Ae_j||}$

In [195]:
def find_simillar_documents_beta(query, term_by_document_matrix, terms, documents, k = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))
    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * idfs[i]
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    
    if q_norm == 0:
        return []
    
    similarities = []
    
    for j in range(M):
        d_j = term_by_document_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        similarities.append(((q_T * d_j)[0,0] / (q_norm * d_j_norm), documents[j]))

    return sorted(similarities, key = lambda x: x[0], reverse = True)[:k]

#### Po uruchomieniu poniższej komórki, pojawi się pod nią pole tekstowe. Należy w nie wpisać zapytanie q (sekwencja słów).

In [198]:
query = input("Podaj zapytanie: ")

Podaj zapytanie: Black people in the japan


In [199]:
similar_documents = find_simillar_documents_beta(query, idf_matrix, ax_Y, ax_X)

for _, document in similar_documents:
    print(document)

page2.txt
page0.txt
page6.txt
page8.txt
page915.txt
page354.txt
page3.txt
page881.txt
page141.txt
page337.txt
page1339.txt
page338.txt
page340.txt
page1022.txt
page375.txt
page36.txt
page1125.txt
page5.txt
page35.txt
page355.txt


---
### 7. Zastosowanie normalizacji wektorów $d_j$ i wektora $q$

In [200]:
def find_simillar_documents(query, term_by_document_matrix, terms, documents, k = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))

    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * idfs[i]

    # normalize d_j
    for i in range(M):
        norm = sparse.linalg.norm(term_by_document_matrix[:,i])
        term_by_document_matrix[:,i] /= norm
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    # normalize q
    q_T /= q_norm
    
    if q_norm == 0:
        return []
    
    similarities = q_T * term_by_document_matrix
    result = [(similarities[0,i], documents[i]) for i in range(M)]

    return sorted(result, key = lambda x: x[0], reverse = True)[:k]

In [201]:
similar_documents = find_simillar_documents(query, idf_matrix, ax_Y, ax_X)

for _, document in similar_documents:
    print(document)

page2.txt
page0.txt
page6.txt
page8.txt
page915.txt
page354.txt
page3.txt
page881.txt
page141.txt
page337.txt
page1339.txt
page338.txt
page340.txt
page1022.txt
page375.txt
page36.txt
page1125.txt
page5.txt
page35.txt
page355.txt


---
### 8. W celu usunięcia szumu z macierzy A zastosujemy SVD i *low rank approximation*

In [202]:
def find_simillar_documents_svd(query, term_by_document_matrix, terms, documents, k, limit = 20):
    q_bag = get_bag_of_words_from_text(query)
    N = len(terms)
    M = len(documents)
    q_vector = sparse.lil_matrix((N, 1))
    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * idfs[i]
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    
    if q_norm == 0:
        return []
    
    U, S, V_t = sparse.linalg.svds(idf_matrix, k)

    Uk = sparse.lil_matrix(U[:, :k])
    Sk = sparse.lil_matrix(np.diag(S[:k]))
    Vk = sparse.lil_matrix(V_t[:k, :])
    filtered_matrix = sparse.csc_matrix(Uk * Sk * Vk)
    
    similarities = []
    
    for j in range(M):
        d_j = filtered_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        similarities.append(((q_T * d_j)[0,0] / (q_norm * d_j_norm), documents[j]))

    return sorted(similarities, key = lambda x: x[0], reverse = True)[:limit]

#### sprawdzenie wyników wyszukiwania z redukcją szumu dla k = 10

In [203]:
k = 10

similar_documents = find_simillar_documents_svd(query, idf_matrix, ax_Y, ax_X, k)

for _, document in similar_documents:
    print(document)

page6.txt
page172.txt
page32.txt
page2.txt
page170.txt
page39.txt
page33.txt
page0.txt
page1033.txt
page30.txt
page8.txt
page125.txt
page130.txt
page56.txt
page31.txt
page52.txt
page1376.txt
page180.txt
page29.txt
page341.txt


---
### 9. Porównamy teraz wyniki dla różnych wartości k oraz wyszukiwania bez redukcji szumu. Spróbujemy znaleźć k, dla którego wyniki wyszukiwania są najlepsze.

In [204]:
k = 100

similar_documents = find_simillar_documents_svd(query, idf_matrix, ax_Y, ax_X, k)

for _, document in similar_documents:
    print(document)

page2.txt
page6.txt
page8.txt
page0.txt
page3.txt
page5.txt
page755.txt
page354.txt
page7.txt
page1022.txt
page4.txt
page744.txt
page489.txt
page358.txt
page1042.txt
page35.txt
page341.txt
page66.txt
page533.txt
page375.txt


In [205]:
k = 200

similar_documents = find_simillar_documents_svd(query, idf_matrix, ax_Y, ax_X, k)

for _, document in similar_documents:
    print(document)

page6.txt
page2.txt
page0.txt
page8.txt
page3.txt
page755.txt
page5.txt
page1022.txt
page358.txt
page354.txt
page35.txt
page7.txt
page744.txt
page915.txt
page489.txt
page983.txt
page338.txt
page340.txt
page533.txt
page344.txt


In [206]:
k = 400

similar_documents = find_simillar_documents_svd(query, idf_matrix, ax_Y, ax_X, k)

for _, document in similar_documents:
    print(document)

page2.txt
page6.txt
page0.txt
page8.txt
page3.txt
page354.txt
page915.txt
page1022.txt
page35.txt
page755.txt
page358.txt
page340.txt
page881.txt
page338.txt
page337.txt
page375.txt
page1134.txt
page534.txt
page5.txt
page36.txt


#### Subiektywnie oceniam, że dla k=200 otrzymałem najlepsze wartości. Tekst page6.txt znajdujący się na 1 miejscu w wynikach jest rzeczywiście tekstem, którego szukałem.

---
### Podsumowanie

Stwórzmy zatem gotowy silnik wyszukiwarki, który zwróci nam oczekiwane wyniki

In [207]:
k = 200
terms = ax_Y
documents = ax_X
U, S, V_t = sparse.linalg.svds(idf_matrix, k)

Uk = sparse.lil_matrix(U[:, :k])
Sk = sparse.lil_matrix(np.diag(S[:k]))
Vk = sparse.lil_matrix(V_t[:k, :])
filtered_matrix = sparse.csc_matrix(Uk * Sk * Vk)

In [208]:
N = len(terms)
M = len(documents)

In [209]:
def find_simillar_documents_final(query, limit = 20):
    q_bag = get_bag_of_words_from_text(query)

    q_vector = sparse.lil_matrix((N, 1))
    for i in range(N):
        q_vector[i, 0] = q_bag[terms[i]] * idfs[i]
    
    q_norm = sparse.linalg.norm(q_vector)
    q_T = q_vector.transpose()
    
    if q_norm == 0:
        return []
    
    similarities = []
    
    for j in range(M):
        d_j = filtered_matrix[:,j]
        d_j_norm = sparse.linalg.norm(d_j)
        similarities.append(((q_T * d_j)[0,0] / (q_norm * d_j_norm), documents[j]))

    return sorted(similarities, key = lambda x: x[0], reverse = True)[:limit]
    

In [211]:
query = input("Podaj zapytanie: ")

Podaj zapytanie: Black people in japan


In [212]:
similar_documents = find_simillar_documents_final(query)

for _, document in similar_documents:
    print(document)

page6.txt
page2.txt
page0.txt
page8.txt
page3.txt
page755.txt
page5.txt
page1022.txt
page358.txt
page354.txt
page35.txt
page7.txt
page744.txt
page915.txt
page489.txt
page983.txt
page338.txt
page340.txt
page533.txt
page344.txt
